In [5]:
%cd ~/lab/stage
from src.dataset import *
from src.problems.two_dimensional.representation import NOmegaPointsScaleBasedPeriodic

nans = lambda df: df[df.isnull().any(axis=1)]


def convert_rotational_eom_to_solution_series(ds, init_theta, n_t_samples, reference_time, save_parquet=True, which={'omega'}):
    mapping = NOmegaPointsScaleBasedPeriodic(n_samples=n_t_samples, n_design_points=3, init_theta=init_theta)
    
    df = ds.get_df()
    x_col = list(filter(lambda x: 'x' in x, df.columns))
    f_col = list(filter(lambda x: 'f' in x, df.columns))

    x_df = ds.get_df().loc[:,x_col]
    f_df = ds.get_df().loc[:,f_col]
    
    idx_f_nan = list(nans(f_df).index)

    n_t_samples = n_t_samples
    n_samples = x_df.index.size
    mapping.reference_time = reference_time

    alpha_samples = np.zeros((n_samples,n_t_samples))
    omega_samples = np.zeros((n_samples,n_t_samples))
    theta_samples = np.zeros((n_samples,n_t_samples))

    for idx, row in x_df.iterrows():
        print(idx) if idx%100000==0 else None
        
        alpha_sample, omega_sample, theta_sample = mapping(row.values)
        
        if 'alpha' in which:
            alpha_samples[idx,:] = alpha_sample
        if 'omega' in which:
            omega_samples[idx,:] = omega_sample
        if 'theta' in which:
            theta_samples[idx,:] = theta_sample

    time_column = [f"t{i}" for i in range(n_t_samples)]
    dataframe_alpha = pd.DataFrame(data=alpha_samples, columns=time_column)
    dataframe_omega = pd.DataFrame(data=omega_samples, columns=time_column)
    dataframe_theta = pd.DataFrame(data=theta_samples, columns=time_column)
    
    dataframe_alpha = dataframe_alpha.drop(idx_f_nan)
    dataframe_omega = dataframe_omega.drop(idx_f_nan)
    dataframe_theta = dataframe_theta.drop(idx_f_nan)

    if 'alpha' in which:
        ds_alpha = DataSetFX(output=f_df.values, input=dataframe_alpha.values, name=ds.name+"_series_alpha", root_dir="data")
    if 'omega' in which:
        ds_omega = DataSetFX(output=f_df.values, input=dataframe_omega.values, name=ds.name+"_series_omega", root_dir="data")
    if 'theta' in which:
        ds_theta = DataSetFX(output=f_df.values, input=dataframe_theta.values, name=ds.name+"_series_theta", root_dir="data")
        
    _return = []
    if save_parquet:
        if 'alpha' in which:
            ds_alpha.to_parquet(file_name=ds.name+"_series_alpha", root_dir="data")
            _return.append(ds_alpha)
        if 'omega' in which:
            ds_omega.to_parquet(file_name=ds.name+"_series_omega", root_dir="data")
            _return.append(ds_omega)
        if 'theta' in which:
            ds_theta.to_parquet(file_name=ds.name+"_series_theta", root_dir="data")
            _return.append(ds_theta)
        
    return _return


nans = lambda df: df[df.isnull().any(axis=1)]

def convert_rotational_eom_to_omega_series(ds, init_theta, n_t_samples, reference_time, save_parquet=True):
    mapping = NOmegaPointsScaleBasedPeriodic(n_samples=n_t_samples, n_design_points=3, init_theta=init_theta)
    df = ds.get_df().dropna().reset_index(drop=True)
    
    x_col = list(filter(lambda x: 'x' in x, df.columns))
    f_col = list(filter(lambda x: 'f' in x, df.columns))

    x_df = df.loc[:,x_col]
    f_df = df.loc[:,f_col]
    
    n_t_samples = n_t_samples
    n_samples = x_df.index.size
    mapping.reference_time = reference_time

    omega_samples = np.zeros((n_samples,n_t_samples))

    for idx, row in x_df.iterrows():
        print(idx) if idx%100000==0 else None
        alpha_sample, omega_sample, theta_sample = mapping(row.values)
        omega_samples[idx,:] = omega_sample

    time_column = [f"t{i}" for i in range(n_t_samples)]
    dataframe_omega = pd.DataFrame(data=omega_samples, columns=time_column)
    ds_omega = DataSetFX(output=f_df.values, input=dataframe_omega.values, name=ds.name+"_series_omega", root_dir="data")

    _return = []
    if save_parquet:

        ds_omega.to_parquet(file_name=ds.name+"_series_omega", root_dir="data")
        
    return ds_omega

#https://open.spotify.com/track/1ibeKVCiXORhvUpMmtsQWq?si=fVJAVtOtTHmZm0KPglam-w

#https://open.spotify.com/track/1ibeKVCiXORhvUpMmtsQWq?si=fVJAVtOtTHmZm0KPglam-w

/home/ggarrett/lab/stage


In [6]:
ds = DataSetFX.from_parquet(root_dir="data", file_name="dataSet2_haltonSampled_6_sdp5")

# df = ds.get_df()
# df = df.sample(frac=0.5)
# ds = DataSetFX(input=df.loc[:,filter(lambda x: 'x' in x, df.columns)].values,
#                output=df.loc[:,filter(lambda x: 'f' in x, df.columns)].values,
#                root_dir="data/AttitudeTrajectoryProblem2D_C1_R1", file_name="dataSet2_haltonSampled_6_sdp5.parquet")

# ds.to_parquet(file_name=ds.name, root_dir='data')

reference_time = 6529.276071694349
n_t_samples = 652
init_theta = -3.141592653589793

ds_omega = convert_rotational_eom_to_omega_series(ds, init_theta, n_t_samples, reference_time)

# nans = lambda df: df[df.isnull().any(axis=1)]
# nans(ds_omega.get_df())

0
100000


### Args:
- in_channels (int): Number of channels in the input image
- out_channels (int): Number of channels produced by the convolution
- kernel_size (int or tuple): Size of the convolving kernel
- stride (int or tuple, optional): Stride of the convolution. Default: 1
- padding (int or tuple, optional): Zero-padding added to both sides of the input. Default: 0
- padding_mode (string, optional). Accepted values `zeros` and `circular` Default: `zeros`
- dilation (int or tuple, optional): Spacing between kernel elements. Default: 1
- groups (int, optional): Number of blocked connections from input channels to output channels. 

In [ ]:
print(ds.get_df())
print(ds.get_df().dropna().reset_index(drop=True))

from src.surrogate.modules import *
from src.surrogate.deeplearning.core import *

### Test case 1

transform

Data:
- F = F, X = Theta
- 



In [ ]:
from sklearn import preprocessing
from torch import nn

tf1_1 = preprocessing.MinMaxScaler(feature_range=(0, 1))
tx1_1 = preprocessing.StandardScaler()

tf1_2 = preprocessing.MinMaxScaler(feature_range=(0, 1))
tx1_2 = preprocessing.StandardScaler()

tf1_3 = preprocessing.MinMaxScaler(feature_range=(0, 1))
tx1_3 = preprocessing.StandardScaler()

transform_1 = TransformDataFrame.from_ordered_dict(OrderedDict([
    ("tf1", (tf1_1, ["f1", "f2", "f3", "f4", 'f5'])),
    ("tx1", (tx1_1, [f"x{i}" for i in range(1,653)]))
#     ("tx1", (tx2, [f"x{i}" for i in range(652)])),
]
))

transform_2 = TransformDataFrame.from_ordered_dict(OrderedDict([
    ("tf1", (tf1_2, ["f1", "f2", "f3", "f4", 'f5'])),
    ("tx1", (tx1_2, [f"x{i}" for i in range(1,653)]))
#     ("tx1", (tx2, [f"x{i}" for i in range(652)])),
]
))
transform_3 = TransformDataFrame.from_ordered_dict(OrderedDict([
    ("tf1", (tf1_3, ["f1", "f2", "f3", "f4", 'f5'])),
    ("tx1", (tx1_3, [f"x{i}" for i in range(1,653)]))
#     ("tx1", (tx2, [f"x{i}" for i in range(652)])),
]
))
# NB10422645 (Stage - Airbus)/data/
ds_t1_omega = DataSetFSeries.from_parquet(root_dir="data", file_name="dataSet2_randomSampled_6_series_omega", transform=transform_1)
ds_t2_omega = DataSetFSeries.from_parquet(root_dir="data", file_name="dataSet2_haltonSampled_6_series_omega", transform=transform_2)
ds_t3_omega = DataSetFSeries.from_parquet(root_dir="data", file_name="dataSet2_gridSampled_6_series_omega", transform=transform_3)

# Define training function.
def train_model(name, ds, model, model_folder, lr=1e-1, epoch=50, load=False, scheduler=None):
    trainer = Trainer(model_folder, ds, model, torch.optim.Adam, torch.nn.MSELoss, num_epochs=epoch, batch_size=int(1e2), learning_rate=lr)
    if scheduler is not None:
        trainer.add_scheduler(torch.optim.lr_scheduler.MultiStepLR, milestones=scheduler[0], gamma=scheduler[1])
    if load:
        trainer.load_checkpoint(model_folder)
    trainer.train()
    return trainer._model

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
    
class Flatten2(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), 1, -1)
    
class Unflatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), input.size(1), 1, 1)

In [ ]:
import torch
print(torch.cuda.is_available())
name = "experiment1_series"
folder = f"models/{name}"

# ds = ds_t1_omega
# print(ds.get_df())

# model1 = nn.Sequential(
#     BasicBlock(input_size=1, output_size=50, kernel_size=(1, 10), stride=2),
#     torch.nn.MaxPool2d((1, 50), stride=1, padding=0, dilation=1, return_indices=False, ceil_mode=False),
# #     ResidualBlock(hidden_size=50, kernel_size=(1,10), stride=1),
# #     BasicBlock(input_size=20, output_size=10, kernel_size=(1, 20), stride=1),
# #     torch.nn.MaxPool2d((1, 10), stride=1, padding=0, dilation=1, return_indices=False, ceil_mode=False),
# #     BasicBlock(input_size=10, output_size=5, kernel_size=(1, 10), stride=1),
    
#     Flatten(),
#     nn.Conv2d(500, 50, 1),
#     activation_dict["leaky_relu"],
#     RegressionOutput(50, 4),
    
# )

model1 = nn.Sequential(
    
    nn.Conv2d(1, 64, stride=4, kernel_size=(1,36), padding=0),   # [64,   1, 155]
    nn.LeakyReLU(negative_slope=0.01, inplace=True),             # 
    nn.MaxPool2d((1,11), stride=2, padding=(0,2)),               # [64,   1,  73]
    
    
    nn.Conv2d(64, 128, stride=1, kernel_size=(1,19), padding=9), # [128,  1,  73]
    nn.LeakyReLU(negative_slope=0.01, inplace=True),             # 
    nn.MaxPool2d((1,4), stride=3, padding=(0,2)),                # [128,  1,  24]
    
    nn.Conv2d(128,172, stride=1, kernel_size=(1,3), padding=1),  # [172,  1,  24]
    nn.LeakyReLU(negative_slope=0.01, inplace=True),             # 
    nn.MaxPool2d((1,4), stride=2, padding=(0,2)),                # [172,  1,  11]
    
    nn.Conv2d(172,128, stride=1, kernel_size=(1,3), padding=1),  # [128,  1,  11]
    nn.LeakyReLU(negative_slope=0.01, inplace=True),             # 
    nn.MaxPool2d((1,4), stride=2, padding=(0,2)),                # [128,  1,   5]
    
    Flatten(),
    
    nn.Linear(4096, 4096),                                         # [,860]
    nn.LeakyReLU(negative_slope=0.01, inplace=True),             # 
    
    nn.Linear(4096, 4096),                                         # [,860]
    nn.LeakyReLU(negative_slope=0.01, inplace=True),             # 
    
    nn.Linear(4096, 4),
    Unflatten()
    )


class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
    
class Flatten2(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), 1, -1)
    
class Unflatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), input.size(1), 1, 1)


model2 = nn.Sequential(
    
    Flatten2(),
    
    nn.Conv1d(1, 126, stride=4, kernel_size=72, padding=0),       # [64,   1, 155]
    nn.LeakyReLU(negative_slope=0.01, inplace=True),             # 
    nn.MaxPool1d(11, stride=2, padding=0),                       # [64,   1,  73]
    
    nn.Conv1d(126, 258, stride=1, kernel_size=19, padding=9),     # [128,  1,  73]
    nn.LeakyReLU(negative_slope=0.01, inplace=True),             # 
    nn.MaxPool1d(4, stride=3, padding=0),                        # [128,  1,  24]
    
    nn.Conv1d(258, 258, stride=1, kernel_size=3, padding=1),     # [172,  1,  24]
    nn.LeakyReLU(negative_slope=0.01, inplace=True),             # 
    nn.MaxPool1d(4, stride=2, padding=0),                        # [172,  1,  11]
    
    nn.Conv1d(258,126, stride=1, kernel_size=3, padding=1),      # [128,  1,  11]
    nn.LeakyReLU(negative_slope=0.01, inplace=True),             # 
    nn.MaxPool1d(4, stride=2, padding=0),                        # [128,  1,   5]
    
    Flatten(),
    
    nn.Linear(504, 504),                                       # [,860]
    nn.LeakyReLU(negative_slope=0.01, inplace=True),             # 
    
    nn.Linear(504, 504),                                       # [,860]
    nn.LeakyReLU(negative_slope=0.01, inplace=True),             # 
    
    nn.Linear(504, 5),
    Unflatten()
    )

print(model2)

In [ ]:
model = train_model("dataSet2_randomSampled_6_", ds_t1_omega, model2, folder, load=True,
           scheduler=([50], 0.1),
           epoch=100,
           lr=1e-4)
# https://open.spotify.com/track/0rUNZQuYQvOz6A6zwyT6tM?si=6WSixNxgSfW2FTLLytNB8g

In [ ]:
model = train_model("dataSet2_haltonSampled_6_", ds_t2_omega, model2, folder, load=False,
           scheduler=([50], 0.1),
           epoch=100,
           lr=1e-4)

In [ ]:
model = train_model("dataSet2_gridSampled_6_", ds_t2_omega, model2, folder, load=False,
           scheduler=([50], 0.1),
           epoch=100,
           lr=1e-4)

In [ ]:
x_ = list(model.children())[1].weight[0]
print(len(x_.flatten()))
print(list(model.children())[1].weight[0])

In [ ]:
model.eval()

# Fixing random state for reproducibility
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
count = 0
for res in list(model.children())[1].weight[::1]:
# res = .weight[0]
    res2 = res.detach().cpu().numpy().flatten()
    plt.figure(dpi=400, figsize=(10,2))
    plt.axis('off')
    imgplot = plt.imshow(np.tile(res2.reshape(1,-1),(1,1)), cmap='gray')



In [ ]:
print(list(model.children())[4].weight.shape)


# for res in list(model.children())[4].weight[::2]:
# # res = .weight[0]
#     res2 = res.detach().cpu().numpy().flatten()
#     plt.figure(dpi=400, figsize=(10,2))
#     plt.axis('off')
#     imgplot = plt.imshow(np.tile(res2.reshape(1,-1),(20,1)), cmap='gray')

res = list(model.children())[14].weight.detach().cpu().numpy()[:,:,]

# avg = np.mean(res, axis=2)
print(res.shape)

plt.figure(dpi=500, figsize=(10,2))
plt.axis('off')
# imgplot = plt.imshow(avg[np.mean(avg,axis=1).argsort()], cmap='gray')
imgplot = plt.imshow(res, cmap='gray')


print(avg.shape)

In [ ]:
for res in avg:
# res = .weight[0]
#     res2 = res.detach().cpu().numpy().flatten()
    plt.figure(dpi=400, figsize=(10,2))
    plt.axis('off')
    imgplot = plt.imshow(np.tile(res.reshape(1,-1),(1,1)), cmap='gray')



In [ ]:
for res in list(model.children())[10].weight[::3]:
# res = .weight[0]
    res2 = res.detach().cpu().numpy().flatten()
    plt.figure(dpi=400, figsize=(10,2))
    plt.axis('off')
    imgplot = plt.imshow(np.tile(res2.reshape(1,-1),(1,1)), cmap='gray')

